In [2]:
:opt no-lint

#  22 Reader
## 22.1 Reader
`Reader` address the need to access global data. Passing such data as arguments would negatively impact code readability.

## 22.2 A new beginning
- The `Functor` of functions is function composition.
- The `Applicative` and `Monad` of functions chain the argument forward before compositing (since they are variants of `Functor`, they retain core `Functor` functionality which is composition)

`Reader` is a way of abstracting out function application, and it gives us a way to do computation in terms
of an argument that hasn’t been supplied yet.
### Short Exercise: Warming up

In [27]:
import Data.Char

cap :: String -> String
cap = map toUpper

rev :: String -> String
rev = reverse

composed :: String -> String
composed = cap . rev
composed "Julie"

fmapped :: String -> String
fmapped = fmap cap rev
fmapped "Chris"

tupled :: String -> (String, String)
tupled = (,) <$> cap <*> rev
tupled "Julie"

tupled' :: String -> (String, String)
tupled' = do
  c <- cap
  r <- rev
  pure (c, r)
tupled' "Julie"

tupled'' :: String -> (String, String)
tupled'' = cap >>= (\c -> rev >>= (\r -> pure (c, r)))
tupled'' "Julie"

"EILUJ"

"SIRHC"

("JULIE","eiluJ")

("JULIE","eiluJ")

("JULIE","eiluJ")

## 22.3 This is Reader
`Reader`is based on function composition, because it lets us compose functions without explicitly having to recognize the argument that will eventually arrive.
The term _Reader_, refers to the `Monad` instance.
## 22.4 Breaking down the Functor of functions
The instance is defined as:
```haskell
instance Functor ((->) r)
```
Since `(->)` as a kindness of `* -> * -> *`, we can only lift a partial applied function, where `r` is the input argument of the function.
The composition based implementation can be verified with types as follow:
```haskell
.    ::              (b -> c) -> (a -> b) -> (a -> c)
fmap :: Functor f => (b -> c) ->     f b  ->     f c
```
Hence, `f` is `((->) a)`
## 22.5 But uh, Reader?
`Reader` is a newtype wrapper for the function type:

In [28]:
newtype Reader r a = Reader { runReader :: r -> a }

Its `Functor` instance is:

In [30]:
{-# LANGUAGE InstanceSigs #-}

instance Functor (Reader r) where
  fmap :: (a -> b) -> Reader r a -> Reader r b
  fmap f (Reader ra) = Reader $ f . ra

### Exercise: Ask
Implement the following function.

In [31]:
ask :: Reader a a
ask = Reader id

## 22.6 Functions have an Applicative, too
The term _Reader_ may sometimes refers to the `Applicative` or `Monad` associated with the partially applied function type, rather than to `Reader`.
### Exercise: Reading comprehension
1. Write `liftA2` yourself.

In [32]:
myLiftA2 :: Applicative f => (a -> b -> c) -> f a -> f b -> f c
myLiftA2 abc fa fb = abc <$> fa <*> fb

2. Write the following function.

In [33]:
asks :: (r -> a) -> Reader r a
asks = Reader

3. Implement the `Applicative` for `Reader`.

In [34]:
{-# LANGUAGE InstanceSigs #-}

instance Applicative (Reader r) where
  pure :: a -> Reader r a
  pure a = Reader $ const a

  (<*>) :: Reader r (a -> b) -> Reader r a -> Reader r b
  (Reader rab) <*> (Reader ra) = Reader $ \r -> rab r (ra r)

## 22.7 The Monad of functions

The structure is `((->) r)`, just like with `Functor` and `Applicative` instances.
### Exercise: Reader Monad
1. Implement the `Reader` `Monad`.

In [35]:
{-# LANGUAGE InstanceSigs #-}

instance Monad (Reader r) where
  return = pure
  (>>=) :: Reader r a -> (a -> Reader r b) -> Reader r b
  (Reader ra) >>= aRb = Reader $ \r -> (runReader $ aRb (ra r)) r

2. Rewrite the monadic `getDogRM` to use your `Reader` datatype.

In [36]:
newtype HumanName = HumanName String deriving (Eq, Show)
newtype DogName = DogName String deriving (Eq, Show)
newtype Address = Address String deriving (Eq, Show)

data Person = Person {humanName :: HumanName, dogName :: DogName, address :: Address} deriving (Eq, Show)
data Dog = Dog {dogsName :: DogName, dogsAddress :: Address} deriving (Eq, Show)

dogNameRM :: Reader Person DogName
dogNameRM = dogName <$> ask

addressRM :: Reader Person Address
addressRM = address <$> ask

getDogRM :: Reader Person Dog
getDogRM = do
  name <- dogNameRM
  addy <- addressRM
  return $ Dog name addy

runReader getDogRM $ Person (HumanName "Sebastian") (DogName "Argo") (Address "Colmar")

Line 15: Use <$>
Found:
do name <- dogNameRM
   addy <- addressRM
   return $ Dog name addy
Why not:
do name <- dogNameRM
   Dog name <$> addressRM

Dog {dogsName = DogName "Argo", dogsAddress = Address "Colmar"}

## 22.8 Reader Monad by itself is boring
Discutere il paragrafo a pagina 865 in basso
## 22.9 You can only change what comes below
??? NON CHIARA LA FRASE ALL'INIZIO
The type argument `r` is read-only

In [2]:
import Control.Monad.Reader

withReaderT :: (r' -> r)
-- ^ The function to modify
-- the environment
  -> ReaderT r m a
-- ^ Computation to run in the
-- modified environment
  -> ReaderT r' m a

withReaderT f m = ReaderT $ runReaderT m . f

-- come si usa??

## 22.10 You tend to see ReaderT, not Reader
`Reader` is typically used in a stack of multiple types providing a `Monad` instance. In that case it's a monad transformer and it's named `ReaderT`
Moreover a `Reader` usually wraps a record of several values.
## 22.11 Chapter exercises
#### A warm-up stretch

In [38]:
import Control.Applicative
import Data.Maybe

x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]

-- zip x and y using 3 as the lookup key
xs :: Maybe Integer
xs = lookup 3 $ zip x y

-- zip y and z using 6 as the lookup key
ys :: Maybe Integer
ys = lookup 6 $ zip y z

-- zip x and y using 4 as the lookup key
zs :: Maybe Integer
zs = lookup 4 $ zip x y

-- zip x and z using a variable lookup key
z' :: Integer -> Maybe Integer
z' n = lookup n $ zip x z

Add the ability to make a `Maybe (,)` of values using `Applicative`.

In [39]:
x1 :: Maybe (Integer, Integer)
x1 = (,) <$> xs <*> ys

x2 :: Maybe (Integer, Integer)
x2 = (,) <$> ys <*> zs

x3 :: Integer -> (Maybe Integer, Maybe Integer)
x3 n = (z' n, z' n)

Make some helper functions.

In [42]:
summed :: Num c => (c, c) -> c
summed = uncurry (+)

bolt :: Integer -> Bool
bolt = (<3)

do
  print $ sequenceA [Just 3, Just 2, Just 1]
  print $ sequenceA [x, y]
  print $ sequenceA [xs, ys]
  print $ summed <$> ((,) <$> xs <*> ys)
  print $ fmap summed ((,) <$> xs <*> zs)
  print $ bolt 7
  print $ fmap bolt z

Just [3,2,1]
[[1,4],[1,5],[1,6],[2,4],[2,5],[2,6],[3,4],[3,5],[3,6]]
Just [6,9]
Just 15
Nothing
False
[False,False,False]

The following combines `sequenceA` and `Reader` in a surprising way:

In [43]:
print $ sequenceA [(>3), (<8), even] 7
  
sequA :: Integral a => a -> [Bool]
sequA = sequenceA [(>3), (<8), even]

s' = summed <$> ((,) <$> xs <*> ys)

[True,True,False]

1. Fold the `Boolean` conjunction operator over the list of results of `sequA` (applied to some value).

In [44]:
print $ and $ sequA 7

False

2. Apply `sequA` to `s'` (you’ll need `fromMaybe`).

In [45]:
print $ sequA $ fromMaybe 0 s'

[True,False,False]

3. Apply `bolt` to `ys` (you’ll need `fromMaybe`).

In [46]:
print $ bolt $ fromMaybe 0 ys

False

#### Rewriting Shawty
Instead of manually passing the database connection `rConn` from `main` to the `app` function, use `ReaderT` to make the database connection
available.

In [49]:
{-# LANGUAGE OverloadedStrings #-}

import Control.Monad (replicateM)
import Control.Monad.IO.Class (liftIO)
import qualified Data.ByteString.Char8 as BC
import Data.Text.Encoding (decodeUtf8, encodeUtf8)
import qualified Data.Text.Lazy as TL
import qualified Database.Redis as R
import Network.URI (URI, parseURI)
import qualified System.Random as SR
import Web.Scotty

alphaNum :: String
alphaNum = ['A'..'Z'] ++ ['0'..'9']

randomElement :: String -> IO Char
randomElement xs = do
  let maxIndex :: Int
      maxIndex = length xs - 1
  -- Right of arrow is IO Int, so randomDigit is Int
  randomDigit <- SR.randomRIO (0, maxIndex) :: IO Int
  return (xs !! randomDigit)

shortyGen :: IO String
shortyGen =
  replicateM 7 (randomElement alphaNum)

saveURI :: R.Connection
        -> BC.ByteString
        -> BC.ByteString
        -> IO (Either R.Reply R.Status)
saveURI conn shortURI uri =
  R.runRedis conn $ R.set shortURI uri

getURI  :: R.Connection
        -> BC.ByteString
        -> IO (Either R.Reply (Maybe BC.ByteString))
getURI conn shortURI = R.runRedis conn $ R.get shortURI

linkShorty :: String -> String
linkShorty shorty =
  concat [ "<a href=\""
         , shorty
         , "\">Copy and paste your short URL</a>"
         ]

shortyCreated :: Show a => a -> String -> TL.Text
shortyCreated resp shawty =
  TL.concat [ TL.pack (show resp)
            , " shorty is: ", TL.pack (linkShorty shawty)
            ]

shortyAintUri :: TL.Text -> TL.Text
shortyAintUri uri =
  TL.concat [ uri
            , " wasn't a url, did you forget http://?"
            ]

shortyFound :: TL.Text -> TL.Text
shortyFound tbs =
  TL.concat ["<a href=\"", tbs, "\">", tbs, "</a>"]

app :: R.Connection -> ScottyM ()
app rConn = do
  get "/" $ do
    uri <- param "uri"
    let parsedUri :: Maybe URI
        parsedUri = parseURI (TL.unpack uri)
    case parsedUri of
      Just _  -> do
        shawty <- liftIO shortyGen
        let shorty = BC.pack shawty
            uri' = encodeUtf8 (TL.toStrict uri)
        resp <- liftIO (saveURI rConn shorty uri')
        html (shortyCreated resp shawty)
      Nothing -> text (shortyAintUri uri)
  get "/:short" $ do
    short <- param "short"
    uri <- liftIO (getURI rConn short)
    case uri of
      Left reply -> text (TL.pack (show reply))
      Right mbBS -> case mbBS of
        Nothing -> text "uri not found"
        Just bs -> html (shortyFound tbs)
          where tbs :: TL.Text
                tbs = TL.fromStrict (decodeUtf8 bs)

appRM :: ReaderT R.Connection ScottyM ()
appRM = ReaderT app
  
main :: IO ()
main = do
  rConn <- R.connect R.defaultConnectInfo
  scotty 3000 $ runReaderT appRM rConn

## 22.12 Definition
A _monad transformer_ is a type that takes a monad as an argument and returns a monad as a result. It allows us to combine two monads into one that shares the behaviors of both.